
***DR-Learner***
* É um meta-algoritmo de Machine Learning (Chernozhukov et al., 2018; Nie & Wager, 2021), que usa a ideia de dupla robustez junto com algoritmos de ML para estimar Conditional Average Treatment Effects (CATE). O DR-Learner aproveita a estrutura AIPW (score de influência duplamente robusto) e depois ajusta modelos de ML em cima disso para obter heterogeneidade do efeito. Portanto, DR-Learner é um desdobramento moderno do AIPW aplicado à estimação de efeitos condicionais (CATE), não sinônimo.
